In [1]:
from datetime import datetime, timedelta, timezone
import pandas as pd
from robo_trader import Portfolio, Interval, DummyExchange
from robo_trader.strategies import LorentzianStrategy, LorentzianSettings
from robo_trader.price_providers import YFinancePriceProvider

In [5]:
SYMBOL = "BTC-GBP"
inception_date = datetime(2000, 1, 1, tzinfo=timezone.utc)
start_date = datetime(2023, 7, 1, tzinfo=timezone.utc)
end_date = datetime(2023, 8, 1, tzinfo=timezone.utc)
interval = Interval.ONE_HOUR
period = 24

settings = LorentzianSettings(
    neighborsCount = 8, maxBarsBack = 2000, useDynamicExits = True,
    useEmaFilter = False, emaPeriod = 200,
    useSmaFilter = False, smaPeriod = 200,
    useKernelSmoothing = True, lookbackWindow = 8, relativeWeight = 8.0, regressionLevel = 25, crossoverLag = 2,
    useVolatilityFilter = True,
    useRegimeFilter = True, regimeThreshold = -.1,
    useAdxFilter = False, adxThreshold = 25,
    use_RSI = True, RSI_param1 = 14, RSI_param2 = 2,
    use_WT = True, WT_param1 = 10, WT_param2 = 11,
    use_CCI = True, CCI_param1 = 20, CCI_param2 = 2,
    use_ADX = True, ADX_param1 = 20, ADX_param2 = 2,
    use_MFI = True, MFI_param1 = 14
)

price_provider = YFinancePriceProvider()
exchange = DummyExchange({SYMBOL: 100}, 0.004)
portfolio = Portfolio(SYMBOL, exchange, inception_date = inception_date)
strategy = LorentzianStrategy(SYMBOL, price_provider, interval, portfolio, settings)

perf = strategy.backtest(start_date = start_date, end_date = end_date, period = period)

In [6]:
# Set display options
pd.set_option('display.max_rows', None)  # Adjust None to your specific needs
pd.set_option('display.max_columns', None)  # Adjust None as needed
pd.set_option('display.width', 1000)  # Adjust the width to fit your notebook

date_format = lambda x: x.strftime('%Y-%m-%d %H:%M')
perf = perf.style.format({'start_date': date_format, 'end_date': date_format})

display(perf)

,start_date,end_date,price_start,price_end,valuation_start,valuation_end,asset_return,portfolio_return
0,2023-07-01 00:00,2023-07-01 23:00,23985.566406,24070.748047,1000.000000,1000.000000,0.003551,0.000000
1,2023-07-02 00:00,2023-07-02 23:00,24070.748047,24128.527344,1000.000000,1000.000000,0.002400,0.000000
2,2023-07-03 00:00,2023-07-03 23:00,24128.527344,24548.054688,1000.000000,1000.000000,0.017387,0.000000
3,2023-07-04 00:00,2023-07-04 23:00,24548.054688,24199.220703,1000.000000,1000.000000,-0.014210,0.000000
4,2023-07-05 00:00,2023-07-05 23:00,24199.220703,24015.685547,1000.000000,1000.000000,-0.007584,0.000000
5,2023-07-06 00:00,2023-07-06 23:00,24015.685547,23490.406250,1000.000000,1000.000000,-0.021872,0.000000
6,2023-07-07 00:00,2023-07-07 23:00,23490.406250,23628.220703,1000.000000,1000.000000,0.005867,0.000000
7,2023-07-08 00:00,2023-07-08 23:00,23628.220703,23594.484375,1000.000000,1000.000000,-0.001428,0.000000
8,2023-07-09 00:00,2023-07-09 23:00,23594.484375,23497.773438,1000.000000,1000.000000,-0.004099,0.000000
9,2023-07-10 00:00,2023-07-10 23:00,23497.773438,23631.265625,1000.000000,981.903850,0.005681,-0.018096
